# Explanation of the contents in the 'llm' folder

Contents:

1. __init__.py
2. llm.py
3. core.py
4. message.py
5. messages.py
6. groq.py


1. __init__.py

The following code has 2 functions: It serves as an initializer for the packages and defines the public interface of the llm module.

1. **Imports**: Here, we import the classes and functions from the various modules within the 'llm' package such as 'LLM', 'Groq', 'LLMType', 'Message', etc.
2. **__all__**: This is the list which contains the list of all the names of the objects, in this case, the modules. This allows us to much control over the modules.

In [ ]:
# llm/__init__.py

from llm.core import LLM
from llm.groq import Groq
from llm.llm import LLMType
from llm.message import Message, Messages
from llm.messages import transform_messages

__all__ = [
    "LLM",
    "Groq",
    "LLMType",
    "Message",
    "Messages",
    "transform_messages",
]


2. llm/llm.py

Defines the 'LLMType' enumeration which repesents the various types of LLMs (in the event we want to use other LLMs like Llamas or Mistrals.)

**BASE**, **INSTRUCT**, **NEITHER**: These values represent different types of LLMs that can be used, such as a base model, an instruction-tuned model, or neither.

In [ ]:
# llm/llm.py

from enum import Enum

__all__ = ["LLMType"]


class LLMType(Enum):
    BASE = 0
    INSTRUCT = 1
    NEITHER = 2


3. core.py

Here, we define the abstract base class (LLM), which is the foundation of the specific LLM implementations

**LLM Class**: 
1. Constructor which initializes the LLM with a name and optional system prompt and type.
2. 'set_system_prompt' method: method to set or update system prompt
3. Abstract methods - 'chat' and 'complete': abstract methods that must be used in any subclass. These define how the LLM handles chat and text completion tasks.
4. 'generate' method: method to choose whether to use 'chat' or 'complete' methods based on the type of LLM. Allowing for easy switching and use between various types of LLMs.
5. __call__: allows the LLM instance to be called as a function (calls the generate method)

In [ ]:
# llm/core.py

from abc import ABC, abstractmethod
from typing import Optional, Union

from llm.llm import LLMType
from llm.message import Messages

__all__ = ["LLM"]


class LLM(ABC):
    def __init__(self, name: str,
                 system_prompt: str = "",
                 type: Optional[Union[LLMType, int]] = LLMType.NEITHER):
        self.name = name
        self.system_prompt = system_prompt

        if isinstance(type, LLMType):
            self.type = type
        elif isinstance(type, int) and 0 <= type <= 2:
            self.type = LLMType(type)
        elif isinstance(type, int):
            raise ValueError(f"Type {type} not recognized.")
        else:
            raise TypeError(f"Value {type} not of type 'LLMType' or 'int'")

    def set_system_prompt(self, system_prompt: str):
        self.system_prompt = system_prompt
        return self

    @abstractmethod
    def chat(self,
             text: Messages,
             max_new_tokens: int = 256,
             temperature: float = 0.1) -> str:
        pass

    @abstractmethod
    def complete(self,
                 text: str,
                 max_new_tokens: int = 256,
                 temperature: float = 0.1) -> str:
        pass

    def generate(self,
                 text: Messages,
                 max_new_tokens: int = 256,
                 temperature: float = 0.1,
                 instruct: Optional[bool] = None) -> str:
        type = None
        if instruct is None:
            if self.type == LLMType.BASE:
                type = LLMType.BASE
            else:
                type = LLMType.INSTRUCT
        elif instruct:
            type = LLMType.INSTRUCT
        else:
            type = LLMType.BASE

        if type == LLMType.INSTRUCT:
            return self.chat(
                text,
                max_new_tokens=max_new_tokens,
                temperature=temperature
            )
        else:
            if not isinstance(text, str):
                raise ValueError("Unsupported type for input 'text'")
            return self.complete(
                text,
                max_new_tokens=max_new_tokens,
                temperature=temperature
            )

    def __call__(self,
                 text: Messages,
                 max_new_tokens: int = 256,
                 temperature: float = 0.1,
                 instruct: Optional[bool] = None) -> str:
        return self.generate(text, max_new_tokens,
                             temperature, instruct)


4. llm/message.py

Defines the structure for chat interactions with the LLM.

**Message class**: 
1. Attributes: defines a message with a role, such as user or system, and content.
2. Message type: Union type which represents different possible formats for messages (list of 'Message' objects, list of dictionaries, or a simple string)

In [2]:
# llm/message.py

from typing import Union
from pydantic import BaseModel

__all__ = [
    "Message",
    "Messages"
]


class Message(BaseModel):
    role: str
    content: str


Messages = Union[
    list[Message],
    list[dict[str, str]],
    str
]

5. llm/messages.py

utility functions for working with messages

1. **transform_messages** function:
   - transforms input messages into a standard list of dictionaries format for compatibility with Groq API. 
   - checks format of input messages, handles various formats such as strings, list of 'Message' objects, list of dictionaries. 
   - Optionally, it inserts a system prompt at the beginning of the message list.

In [ ]:
# llm/messages.py

from llm.message import Message, Messages

__all__ = [
    "transform_messages"
]


def transform_messages(input: Messages, 
                       system_prompt: str = "") -> list[dict[str, str]]:
    messages: list[dict[str, str]]
    if isinstance(input, str):
        messages = [{
            "role": "user",
            "content": input
        }]
    elif isinstance(input, list) and isinstance(input[0], Message):
        messages = [
            dict(msg)
            for msg in input
        ]
    elif isinstance(input, list) and isinstance(input[0], dict):
        messages = input
    
    else:
        raise TypeError("Unsupported format for messages item")
    
    if messages[0]["role"] != "system" and system_prompt is not None and len(system_prompt.strip()) > 0:
            messages.insert(0, {"role": "system", "content": system_prompt})
    
    return messages


    

llm/groq.py

Implementation of the LLM abstract class using the Groq API

- 'Groq' class inherits the 'LLM' class and initializes the Groq client using an API key.
- implements chat functionality using the Groq API which supports chat transformation and token limits
- uses the 'chat' method for text completion by formatting input text in a specific way.

In [ ]:
# llm/groq.py

import groq

from typing import Optional, Union

from llm.message import Messages
from llm.llm import LLMType
from llm.messages import transform_messages
from llm.core import LLM

__all__ = [
    "Groq"
]


class Groq(LLM):
    def __init__(self,
                 model_id: str,
                 api_key: str,
                 system_prompt: str = "",
                 type: Optional[Union[LLMType, int]] = LLMType.NEITHER):
        super().__init__(
            model_id, system_prompt,
            type
        )
        self.client = groq.Groq(api_key=api_key)

    def chat(self,
             text: Messages,
             max_new_tokens: int = 1024,
             temperature: float = 0.1) -> str:
        messages = transform_messages(text, self.system_prompt)

        message = self.client.chat.completions.create(
            max_tokens=max_new_tokens,
            messages=messages,
            temperature=temperature,
            model=self.name
        )
        output = message.choices[0].message.content
        return output

    def complete(self,
                 text: str,
                 max_new_tokens: int = 1024,
                 temperature: float = 0.1) -> str:
        text = f"Continue writing: {text}"
        
        return self.chat(text, max_new_tokens=max_new_tokens, temperature=temperature)
